In [1]:
from py_src.yolov4.tf import YOLOv4

yolo = YOLOv4(tiny=True)

yolo.classes = "test/coco.names"

yolo.input_size = 608
yolo.batch_size = 32
yolo.subdivision = 16

yolo.make_model()
yolo.load_weights("test/yolov4-tiny.weights", weights_type="yolo")

In [2]:
data_set = yolo.load_dataset("test/val2016.txt")

In [3]:
import tensorflow as tf
import numpy as np
import cv2

for i, (images, gt) in enumerate(data_set):
    for j in range(len(images)):
        _candidates = []
        for candidate in gt:
            grid_size = candidate.shape[1]
            _candidates.append(
                tf.reshape(candidate[j], shape=(1, grid_size * grid_size * 3, -1))
            )
        candidates = np.concatenate(_candidates, axis=1)

        frame = images[j, ...] * 255
        frame = frame.astype(np.uint8)

        pred_bboxes = yolo.candidates_to_pred_bboxes(candidates[0])
        pred_bboxes = yolo.fit_pred_bboxes_to_original(pred_bboxes, frame.shape)
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        image = yolo.draw_bboxes(frame, pred_bboxes)
        cv2.namedWindow("result", cv2.WINDOW_AUTOSIZE)
        cv2.imshow("result", image)
        while cv2.waitKey(10) & 0xFF != ord("q"):
            pass
    if i == 10:
        break

cv2.destroyWindow("result")

ValueError: not enough values to unpack (expected 3, got 2)